In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import glob
import numpy as np

sns.set(style="darkgrid", font_scale=1.0)

# path_list = ["./logs_csv/DQN/2x2_10x/", "./logs_csv/DQN/3x3_10x/", "./logs_csv/DQN/4x4_10x/"]
# path_list = ["./logs_csv/A2C/8env/2x2_10x/", "./logs_csv/A2C/8env/3x3_10x/", "./logs_csv/A2C/8env/4x4_10x/"]
# path_list = ["./logs_csv/PPO/8env/2x2_10x/",  "./logs_csv/PPO/8env/3x3_10x/", "./logs_csv/PPO/8env/4x4_10x/"]
# path_list = ["./logs_csv/DQN/2x2_10x/",  "./logs_csv/A2C/8env/2x2_10x/", "./logs_csv/PPO/8env/2x2_10x/"]

# path_list = ["./logs_csv/DQN/2x2_5x/",  "./logs_csv/DQN/2x2_10x/", "./logs_csv/DQN/2x2_15x/"]
# path_list = ["./logs_csv/A2C/8env/2x2_5x/",  "./logs_csv/A2C/8env/2x2_10x/", "./logs_csv/A2C/8env/2x2_15x/"]
path_list = ["./logs_csv/PPO/8env/2x2_5x/",  "./logs_csv/PPO/8env/2x2_10x/", "./logs_csv/PPO/8env/2x2_15x/"]


def get_plot_info(path):
    
    df = pd.concat(map(pd.read_csv, glob.glob(path + "/*.csv")), axis=1)
    df = df.dropna()

    for i in range(len(df["Wall time"].columns)):
        df['Training Time ' + str(i)] = df['Wall time'].iloc[:,i].apply(lambda x: (x - df['Wall time'].iloc[:,i].min())/60)
        df.rename(columns={"Training Time " + str(i): "Training Time"}, inplace=True)

    df_mean=df.groupby(level=0,axis=1).mean()
    df_std=df.groupby(level=0,axis=1).std(numeric_only=True)["Value"]
    df_count = df.groupby(level=0,axis=1).count()["Value"]

    df_ci_upper = df_mean["Value"] + 1.96*df_std/np.sqrt(df_count)
    df_ci_lower = df_mean["Value"] - 1.96*df_std/np.sqrt(df_count)

    return df_mean, df_std, df_ci_upper, df_ci_lower


In [27]:
x_variable = "Step"

for path in path_list:

    df_mean, df_std, df_ci_upper, df_ci_lower = get_plot_info(path)



    # sns.lineplot(data=df3, x="Step", y="Value", errorbar=('ci', 95))
    # sns.lineplot(data=df_mean, x=x_variable, y="Value", legend='brief', label=path[-8:-5])
    sns.lineplot(data=df_mean, x=x_variable, y="Value", legend='brief', label=path[24:-1])
    # sns.lineplot(data=df_mean, x=x_variable, y="Value", legend='brief', label=path[11:14])

    # Standard deviation
    # plt.fill_between(df_mean["Step"], np.array(df_mean["Value"])-np.array(df_std), np.array(df_mean["Value"])+np.array(df_std), alpha=0.2, color='red')
    # plt.fill_between(df_mean[x_variable], np.array(df_ci_lower), np.array(df_ci_upper), alpha=0.1, color='blue')
    plt.fill_between(df_mean[x_variable], np.array(df_ci_lower), np.array(df_ci_upper), alpha=0.2)

# add title and axis labels
plt.title('PPO performance (action_unit_size = 2x2)')
plt.xlabel(x_variable)
plt.ylabel('Average Episode Reward')

if x_variable == "Step":
    # Set the formatter for the x-axis
    ax = plt.gca()  # Get the current axes
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x * 1e-3:.0f}k'))
elif x_variable == "Training Time":
    # Set the formatter for the x-axis
    ax = plt.gca()  # Get the current axes
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x)} min'))

# show the plot
# plt.savefig("./results/PPO_2x2", dpi=300)
plt.show()
